### Load Pickle files from all environments

In [ ]:
import pickle
from pathlib import Path

root_module = Path.cwd()
ext = ".pkl"
name = "sparse_pendulum_ensemble"
load_dir = root_module.parent.joinpath("data/")
file_dir = load_dir.joinpath(name + ext)
data = pickle.load(open(file_dir, "rb"))

### Smoothen Returns and Compute Mean + Confidence Intervals

In [ ]:
import numpy as np
from itertools import product
from scipy.stats import sem

# Get available methods and noise levels to plot
keys_arr = np.array(list((data["returns"].keys())), dtype=object)
agent_types = np.array(list(set(keys_arr[:, 0])), dtype=tuple)
agent_types = agent_types[agent_types.argsort()]
# rearrange to desire order 
order = [1, 3, 0, 2]
agent_types = agent_types[order].tolist()
noise_std = np.sort(list(set(keys_arr[:, 1])))
ensemble_sizes = np.sort(list(set(keys_arr[:, 2].astype(int))))

data["mean_final_returns"] = {}
data["ci_final_returns"] = {}
for type, noise, size in product(agent_types, noise_std, ensemble_sizes):
    idx = (type, noise, str(size))
    final_returns = data["returns"][idx][-1]
    data["mean_final_returns"][idx] = np.mean(final_returns)
    data["ci_final_returns"][idx] = sem(final_returns)

### Plot results

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from ube_mbrl.utils.plot import PARAMS

plt.rcParams.update(PARAMS)
plt.rcParams['axes.formatter.useoffset'] = False

num_cols = len(noise_std)
fig, axes = plt.subplots(nrows=1, ncols=num_cols, figsize=(7.5, 2.0),gridspec_kw={'wspace':0.05,'hspace':0.1})

ax_dict = {}
for noise, ax in zip(noise_std, axes.flatten()):
    ax_dict[noise] = ax 

colors = {}
cmap = plt.get_cmap("tab10")
for i, type in enumerate(agent_types):
    colors[type] = cmap(i)

wd = 0.2
min_level = -70.0
pos = [-3 * wd/2, -wd/2, wd/2, 3*wd/2]
xticks = list(range(1, len(ensemble_sizes) + 1))
xticks_labels = set()
for (i, type), noise, (ensemble_size, xtick) in product(enumerate(agent_types), noise_std, zip(ensemble_sizes, xticks)):
    idx = (type, noise, str(ensemble_size))
    xticks_labels.add(str(ensemble_size))
    ax_dict[noise].bar(
        xtick + pos[i],
        data["mean_final_returns"][idx] - min_level,
        yerr=data["ci_final_returns"][idx],
        width=wd,
        color=colors[type],
        bottom=min_level
    )

patches = []
for type in agent_types:
    if type == "exact_ube_3":
        label = r"\texttt{exact-ube}" + " (ours)"
    elif type == "ensemble":
        label = r"\texttt{ensemble-var}"
    elif type == "none":
        label = r"\texttt{ensemble-mean}"
    else: label = fr"\texttt{{{type}}}"
    patches.append(mpatches.Patch(color=colors[type], label=label))


axes[0].legend(handles=patches, loc = 'lower center', ncol=4, bbox_to_anchor=(1.5, -0.5), frameon=False)

axes[1].yaxis.set_ticklabels([])
axes[2].yaxis.set_ticklabels([])
axes[0].set_ylabel("Return")

labels = np.sort(np.array(list(xticks_labels), dtype=int))
for noise, ax in ax_dict.items():
    ax.set_xticks(ticks=xticks, labels=labels)
    ax.set_title(f"Angle noise std {noise}")
    ax.set_xlabel("Ensemble size")
    ax.set_ylim(bottom=min_level, top=260)

In [ ]:
# Save figures
import os
import pickle
from pathlib import Path
root_module = Path.cwd()
fig_dir = root_module.parent.joinpath(f"figures/sparse_pendulum_ensemble_ablation.pdf")
fig.savefig(fig_dir, bbox_inches="tight", transparent=False)

# License

>Copyright (c) 2023 Robert Bosch GmbH
>
>This program is free software: you can redistribute it and/or modify <br>
>it under the terms of the GNU Affero General Public License as published<br>
>by the Free Software Foundation, either version 3 of the License, or<br>
>(at your option) any later version.<br>
>
>This program is distributed in the hope that it will be useful,<br>
>but WITHOUT ANY WARRANTY; without even the implied warranty of<br>
>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the<br>
>GNU Affero General Public License for more details.<br>
>
>You should have received a copy of the GNU Affero General Public License<br>
>along with this program.  If not, see <https://www.gnu.org/licenses/>.